In [1]:
import numpy as np
import xarray as xr
from pyfunctions import *

In [2]:
# PFT x Biome Combinations
pb_index = {}
pb_index[1] = [2,5,6,7,8]
pb_index[2] = [4,5,6,8,9]
pb_index[3] = [8,9]
pb_index[4] = [1,2,3]
pb_index[5] = [2,3]
pb_index[6] = [2,3]
pb_index[7] = [5,6,7]
pb_index[8] = [5,6,8,9]
pb_index[9] = []
pb_index[10] = [6,7]
pb_index[11] = [8,9]
pb_index[12] = [7,8,9]
pb_index[13] = [2,3,5,6,7,8]
pb_index[14] = [1,2,3] 
pb_index[15] = [1,2,3,5,6,7,8]

np.save('pb_index.npy', pb_index) 

### Observational Target

In [5]:
# CLM-SP PFTxBiome LAI
#read in biome vector and pft vector
whit=xr.open_dataset('whit/whitkey.nc')
f='/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC0000.clm2.h1.2005-02-01-00000.nc'
ds=xr.open_dataset(f)
pfts=ds.pfts1d_itype_veg

#define the pft-x-biome group
pftbiome=xr.DataArray(np.zeros(ds.pft.shape)+np.nan,dims='pft',name='pb')
for i in range(1,17):
    pftbiome[pfts==i]=whit.biome+10*i

lapft=xr.open_dataset('landarea_transient.nc').landarea_pft
a=lapft.groupby('time.year').mean()
asum=a.groupby(pftbiome).sum()
minarea=5e4
asum=asum.where(asum>minarea)

dsp=xr.open_dataset('/glade/scratch/linnia/archive/LAI_SP_ctsm51d115/lnd/hist/LAI_SP_ctsm51d115.clm2.h1.2000-02-01-00000.nc')
dsp['time']=xr.cftime_range('2000',periods=12,freq='MS',calendar='noleap')
sp_lai_amax=amax(dsp.TLAI).sel(year=2000)
sp_lai_amean=amean(dsp.TLAI).sel(year=2000)


def get_SP_pbmean(pft,biome):

    if (pft<12): # annual max
        lai_sp=sp_lai_amax
    else: # annual mean
        lai_sp=sp_lai_amean
    laig=pftgrid(lai_sp,dsp)

    jvals, ivals = get_sparsegrid() # function in pyfunctions.py
    dsp_sg = laig[:,jvals,ivals]

    b = whit.biome
    ix = np.where(b==biome)[0]

    ai = a.mean(dim='year').sel(pft=pft)
    pb = pft*10+biome-10
    asumi = asum.mean(dim='year')[pb]

    da = dsp_sg.isel(pft=pft,gridcell=ix)

    sp_lai_pb = (1/asumi)*(ai.isel(pft=ix).values*da).sum().compute()
    
    return sp_lai_pb.values

In [6]:
# get observational target
nbiomes = 10
npfts = 17
pb_index = np.load('pb_index.npy',allow_pickle='TRUE').item()

SP_LAI = np.empty((npfts,nbiomes))*np.NaN
for p in range(1,16):
    for b in pb_index[p]:
        SP_LAI[p,b] = get_SP_pbmean(p,b)

np.savetxt("CLM-SP_LAI_pbmean.txt", SP_LAI, delimiter=',')        

### Observational Variance

In [7]:
def get_biomeLAI_var(): # this is only for annual max LAI right now (should be annual mean LAI for grasses)
    
    # CLM-SP LAI on sparse grid
    la = get_la()
    SP_sg = get_SP('h0')

    BM_sp = bmean(amax(SP_sg).mean(dim='year'),la).compute()
    
    # load regridded ILAMB datasets (MODIS, AVHRR, AVH15C1)
    ds_ilamb =xr.open_dataset('/glade/work/linnia/ILAMB/ilamb_data/regridded/ILAMB_LAI_regridded.nc')
    Amax_LAI_MODIS = amax(ds_ilamb.MODIS_LAI).mean(dim='year')

    jvals, ivals = get_sparsegrid()

    BM_MODIS = bmean(amax(ds_ilamb.MODIS_LAI[:,jvals,ivals]).mean(dim='year'),la).compute()
    mask = xr.where(BM_MODIS!=0,1,0)
    BM_AVHRR = bmean(amax(ds_ilamb.AVHRR_LAI[:,jvals,ivals]).mean(dim='year'),la).compute()
    BM_AVH15C1 = bmean(amax(ds_ilamb.AVH15C1_LAI[:,jvals,ivals]).mean(dim='year'),la).compute()

    BM_LAI_obs = [BM_MODIS, BM_AVHRR, BM_AVH15C1, BM_sp]
    
    # variance across datasets ( Biome mean lai max )
    obs_var = np.var(BM_LAI_obs,axis=0)
    
    return obs_var

In [8]:
obs_var = get_biomeLAI_var()
np.savetxt("LAImax_biome_var.txt",obs_var, delimiter=',')